# Algoritmo rutas seguras contra el acoso a la mujer en Medellín

In [9]:
import pandas as pd
from queue import PriorityQueue

#### La función *inicioProceso* se encarga de:
 - Lee y carga el csv
 - Convierte el csv en diferentes dataframes
 - Se genera el dataframe 3 el cual contiene los nodos unicos
 - Se genera el dataframe 4 el cual contiene los destinos de cada nodo origen con la distancia
 - Se genera el datafrmaAcoso el cual contiene los destinos de cada nodo origen con el acoso

In [30]:
import pandas as pd
from queue import PriorityQueue

def inicioProceso():
    # Lectura del csv en dataframe
    df = pd.read_csv("calles_de_medellin_con_acoso.csv", sep=';')

    # Se crea un dataframe con 4 campos
    df1 = df[['origin', 'destination', 'length', 'harassmentRisk']]

    # Dataframe solo con campo origen para mas adelante utilizarlo como los nodos
    df3 = df[['origin']]

    # Cantidad de nodos
    nodos = df3.origin.unique()
    df3 = pd.DataFrame(nodos, columns=['origin'])
    df3.insert(0, 'id', df3.index)

    #  escribe los nodos en un txt
    with open("Nodos.txt", "w") as text_file:
        txt = str((df3.to_string()))
        text_file.write(txt + "\r\n")

    # Almacena en un dataframe nuevo los destinos de cada nodo origen
    df4 = (df1.loc[df1['origin'].isin(df3["origin"])])


    #  escribe los Destinos en un txt
    with open("Destinos.txt", "w") as text_file:
        txt = str((df4.to_string()))
        text_file.write(txt + "\r\n")

    # Se convierte la columna destino con el indice de cada nodo unico
    df4['destination'] = df4['destination'].map(df3.set_index('origin')['id'])
    df4['destination'] = df4['destination'].fillna(0).astype(int)

    # Se convierte la columna origen con el indice de cada nodo unico
    df4['origin'] = df4['origin'].map(df3.set_index('origin')['id'])
    df4['origin'] = df4['origin'].fillna(0).astype(int)

    # se convierte la columna distancia en int para el dijkstra
    df4['length'] = df4['length'].fillna(0).astype(int)

    # se crea nuevo datframe para el camino con el menor acoso
    dfAcoso = df4[['origin', 'destination', 'harassmentRisk']]

    dfAcoso['harassmentRisk'] = dfAcoso['harassmentRisk'].fillna(0)
    
    # Se elimina la columna harassmentRisk ya que para el primer dijkstra solo se necesita la distancia
    df4.drop('harassmentRisk', inplace=True, axis=1)    

    return df1,df3,df4,dfAcoso

def agregar(mylist,dataframetoList,dataframe):
    # Se agregan los nodos
    mylist = dataframetoList["id"].tolist()
    # Se agregan las aristas
    for index, row in dataframe.iterrows():
        add_edge(row['origin'], row['destination'],mylist)

    return mylist

#Funcion que crea una lista adyacente a partir de un dataframe
def add_node(node):
    if node not in mylist:
        mylist.append(node)
    else:
        print("Node ", node, " already exists!")

# agrega las aristas
def add_edge(node1, node2,mylist):
    temp = []
    if node1 in mylist and node2 in mylist:
        if node1 not in adj_list:
            temp.append(node2)
            adj_list[node1] = temp

        elif node1 in adj_list:
            temp.extend(adj_list[node1])
            temp.append(node2)
            adj_list[node1] = temp

#La variable peso puede ser distancia o acoso entre los nodos
def CalculoDistancia(dfNodos,dfDestinos):
    g = Graph(len(dfNodos))
    # Se agregan las aristas
    print(dfDestinos.columns)

    for i in range(len(dfDestinos)) :
        g.add_edge2(dfDestinos.iloc[i, 0], dfDestinos.iloc[i, 1],dfDestinos.iloc[i, 2])

    D = g.dijkstra(0)
    
    return D

# Crea el txt con las distancias u acoso, desde el nodo origen "0" hacia al resto de destinos 
# Se debe tener en cuenta que la distancia u acoso son calculados a partir del nodo "0" hasta cada nodo destino

def imprimir(nombre):
    texto = ''
    if nombre == 'Acoso':
        texto = 'Acoso desde el nodo 0 al nodo '
    else:
        texto = 'Distancia desde el nodo 0 al nodo '

    # Se imprime el resultado del algoritmo dijkstra
    with open("Fin" + nombre + ".txt", "w") as text_file:
        for vertex in range(len(D)):
            txt = (texto, str(vertex), "es", str(D[vertex]) )
            txt = str(txt)
            text_file.write(txt + "\r\n")

# Imprime el grafo
def graph(nombre):
    with open("Grafo" + nombre + ".txt", "w") as text_file:
        for node in adj_list:
            txt = (node, " ---> ", [i for i in adj_list[node]])
            txt = str(txt)
            text_file.write(txt + "\r\n")

# Aca inicia el algoritmo de Dijkstra
class Graph:
    def __init__(self, num_of_vertices):
        self.v = num_of_vertices
        self.edges = [[(-1) for i in range(num_of_vertices)]
                    for j in range(num_of_vertices)]
        self.visited = []


    def add_edge2(self, u, v, weight):
        self.edges[u][v] = weight
        self.edges[v][u] = weight

    def dijkstra(self, start_vertex):
        D = {v: float('inf') for v in range(self.v)}
        D[start_vertex] = 0

        pq = PriorityQueue()
        pq.put((0, start_vertex))

        while not pq.empty():
            (dist, current_vertex) = pq.get()
            self.visited.append(current_vertex)

            for neighbor in range(self.v):
                if self.edges[current_vertex][neighbor] != -1:
                    distance = self.edges[current_vertex][neighbor]
                    if neighbor not in self.visited:
                        old_cost = D[neighbor]
                        new_cost = D[current_vertex] + distance
                        if new_cost < old_cost:
                            pq.put((new_cost, neighbor))
                            D[neighbor] = new_cost
        return D

if __name__ == "__main__":
    # df1 dataframe con 4 campos origin,destination,length,harassmentRisk
    # df3 Dataframe solo con campo origen para mas adelante utilizarlo como los nodos
    # df4 Almacena en un dataframe nuevo los destinos de cada nodo origen
    # dfAcoso se crea nuevo dataframe para el camino con el menor acoso
    adj_list = {}
    mylist = []
    df1,df3,df4,dfAcoso = inicioProceso()

    # agrego las aristas del grafo
    agregar(mylist,df3,df4)

    # Se imprime el grafo en un txt
    graph('Distancia')

    # Se imprime la lista adyacente en un txt
    with open("ListAdyacente.txt", "w") as text_file:
        txt = str(adj_list)
        text_file.write(txt + "\r\n")
    
    # Calcula la menor distancia desde el nodo inicial al nodo final
    dijkstraDistancia = CalculoDistancia(df3,df4)
    
    # Crea el nuevo grafo para calcular la ruta con el menor acoso
    adj_list = {}
    mylist = []
    agregar(mylist,df3,dfAcoso)
    
    graph('Acoso')
    
    # # Se imprime la lista adyacente en un txt
    with open("ListAdyacenteAcoso.txt", "w") as text_file:
        txt = str(adj_list)
        text_file.write(txt + "\r\n")
    
    # Calcula la ruta con el menor acoso
    dijkstraAcoso = CalculoDistancia(df3,dfAcoso)

C:\Users\Carlos\AppData\Local\Temp/ipykernel_15328/4184826443.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfAcoso['harassmentRisk'] = dfAcoso['harassmentRisk'].fillna(0)


Index(['origin', 'destination', 'length'], dtype='object')
Index(['origin', 'destination', 'harassmentRisk'], dtype='object')


# Explicación dataframes

**Dataframe df1 con 4 campos origin,destination,length,harassmentRisk**

In [31]:
df1.head()

,origin,destination,length,harassmentRisk
0,"(-75.5728593, 6.2115169)","(-75.5724985, 6.2113756)",42.867,0.526539
1,"(-75.5705202, 6.2106275)","(-75.570427, 6.2105879)",11.204,0.414356
2,"(-75.5705202, 6.2106275)","(-75.5705604, 6.2105262)",12.109,0.526539
3,"(-75.5687719, 6.2099661)","(-75.5688022, 6.2098867)",9.443,0.302173
4,"(-75.5687719, 6.2099661)","(-75.568715, 6.2099443)",6.741,0.302173


**Dataframe df3 con campo origen**

In [32]:
df3.head()

,id,origin
0,0,"(-75.5728593, 6.2115169)"
1,1,"(-75.5705202, 6.2106275)"
2,2,"(-75.5687719, 6.2099661)"
3,3,"(-75.5674348, 6.2094357)"
4,4,"(-75.5666527, 6.2091202)"


**Dataframe df4 con los nuevos nodos destinos de cada nodo origen para el camino con la menor distancia**


In [33]:
df4.head()

,origin,destination,length
0,0,3105,42
1,1,3039,11
2,1,21011,12
3,2,21192,9
4,2,3371,6


**Dataframe dfAcoso para el camino con el menor acoso**

In [34]:
dfAcoso.head()

,origin,destination,harassmentRisk
0,0,3105,0.526539
1,1,3039,0.414356
2,1,21011,0.526539
3,2,21192,0.302173
4,2,3371,0.302173


# Resultado Distancia más corta

 - Crea el txt con las distancias, desde el nodo origen "0" hacia al resto de destinos.
 - Se debe tener en cuenta que la distancia es calculada a partir del nodo "0" hasta cada nodo destino
 -Se muestran los primeros 20 resultados ya que el grafo consta de 27660 nodos

In [35]:
# Se imprime la lista adyacente en un txt
texto = 'Distancia desde el nodo 0 al nodo '
# range(len(dijkstraDistancia)
for vertex in range(20):
    txt = (texto, str(vertex), "es", str(dijkstraDistancia[vertex]) )
    txt = str(txt)
    print(txt)

('Distancia desde el nodo 0 al nodo ', '0', 'es', '0')
('Distancia desde el nodo 0 al nodo ', '1', 'es', '274')
('Distancia desde el nodo 0 al nodo ', '2', 'es', '477')
('Distancia desde el nodo 0 al nodo ', '3', 'es', '632')
('Distancia desde el nodo 0 al nodo ', '4', 'es', '724')
('Distancia desde el nodo 0 al nodo ', '5', 'es', '790')
('Distancia desde el nodo 0 al nodo ', '6', 'es', '954')
('Distancia desde el nodo 0 al nodo ', '7', 'es', '771')
('Distancia desde el nodo 0 al nodo ', '8', 'es', '451')
('Distancia desde el nodo 0 al nodo ', '9', 'es', '533')
('Distancia desde el nodo 0 al nodo ', '10', 'es', '369')
('Distancia desde el nodo 0 al nodo ', '11', 'es', '328')
('Distancia desde el nodo 0 al nodo ', '12', 'es', '288')
('Distancia desde el nodo 0 al nodo ', '13', 'es', '201')
('Distancia desde el nodo 0 al nodo ', '14', 'es', '883')
('Distancia desde el nodo 0 al nodo ', '15', 'es', '812')
('Distancia desde el nodo 0 al nodo ', '16', 'es', '652')
('Distancia desde el nodo 

# Resultado menor acoso

- Crea el txt con el acoso, desde el nodo origen "0" hacia al resto de destinos.
- Se debe tener en cuenta que el acoso es calculado a partir del nodo "0" hasta cada nodo destino
- Se muestran los primeros 20 resultados ya que el grafo consta de 27660 nodos

In [36]:
# Se imprime la lista adyacente en un txt
texto = 'Acoso desde el nodo 0 al nodo '
# range(len(dijkstraDistancia)
for vertex in range(20):
    txt = (texto, str(vertex), "es", str(dijkstraAcoso[vertex]) )
    txt = str(txt)
    print(txt)

('Acoso desde el nodo 0 al nodo ', '0', 'es', '0')
('Acoso desde el nodo 0 al nodo ', '1', 'es', '3.159235875997017')
('Acoso desde el nodo 0 al nodo ', '2', 'es', '3.463953968471632')
('Acoso desde el nodo 0 al nodo ', '3', 'es', '3.093053699793247')
('Acoso desde el nodo 0 al nodo ', '4', 'es', '3.093053699793247')
('Acoso desde el nodo 0 al nodo ', '5', 'es', '3.093053699793247')
('Acoso desde el nodo 0 al nodo ', '6', 'es', '4.4590962804627035')
('Acoso desde el nodo 0 al nodo ', '7', 'es', '4.156923226491678')
('Acoso desde el nodo 0 al nodo ', '8', 'es', '3.159235875997017')
('Acoso desde el nodo 0 al nodo ', '9', 'es', '3.159235875997017')
('Acoso desde el nodo 0 al nodo ', '10', 'es', '2.6326965633308475')
('Acoso desde el nodo 0 al nodo ', '11', 'es', '2.106157250664678')
('Acoso desde el nodo 0 al nodo ', '12', 'es', '2.106157250664678')
('Acoso desde el nodo 0 al nodo ', '13', 'es', '2.106157250664678')
('Acoso desde el nodo 0 al nodo ', '14', 'es', '2.1865345378801715')
('A